# Thông tin chung

**Mục tiêu buổi học**

- Giới thiệu thư viện Beautiful Soup.
- Hướng dẫn cài đặt và sử dụng thư viện Beautiful Soup để thu thập thông tin Web.

**Kiến thức và kỹ năng đạt được**

- Nắm vững và sử dụng được các đối tượng trong thư viện Beautiful Soup.
- Áp dụng cài đặt được các bài tập thực hành.

**Công cụ thực hành**

- Ngôn ngữ lập trình: Python
- Công cụ thực hành: Anaconda, colab

**Thời gian thực hành**: 3 tiết

# Nội dung lý thuyết

**Beautiful Soup**

- Là bộ thư viện thu thập dữ liệu từ các trang HTML, XML.
- Hỗ trợ bộ phân tích cú pháp HTML (html.parser), XML (lxml.parser).
- Đơn giản, dễ sử dụng.

Cài đặt thư viện BeautifulSoup
```python
pip install BeautifulSoup4
```

# Nội dung thực hành

## Lấy thông tin từ mã nguồn HTML tĩnh

In [79]:
from bs4 import BeautifulSoup
html_doc = """<!DOCTYPE html><html><body><p><a id="link1" 
href="www.3schools.com">www.3schools.com</a><a id="link2" 
href="https://developer.mozilla.org">
https://developer.mozilla.org</a></p><p>This is a paragraph.</p><p>
This is another paragraph</p></body></html>"""
#
soup = BeautifulSoup(html_doc, "html.parser")

In [80]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <body>
  <p>
   <a href="www.3schools.com" id="link1">
    www.3schools.com
   </a>
   <a href="https://developer.mozilla.org" id="link2">
    https://developer.mozilla.org
   </a>
  </p>
  <p>
   This is a paragraph.
  </p>
  <p>
   This is another paragraph
  </p>
 </body>
</html>


In [81]:
print(soup.find(id="link1"))

<a href="www.3schools.com" id="link1">www.3schools.com</a>


In [82]:
print(soup.find_all(name="a"))

[<a href="www.3schools.com" id="link1">www.3schools.com</a>, <a href="https://developer.mozilla.org" id="link2">
https://developer.mozilla.org</a>]


In [83]:
p_tag = soup.p
print(p_tag)

<p><a href="www.3schools.com" id="link1">www.3schools.com</a><a href="https://developer.mozilla.org" id="link2">
https://developer.mozilla.org</a></p>


In [84]:
a_tag = p_tag.a
print(a_tag)

<a href="www.3schools.com" id="link1">www.3schools.com</a>


In [85]:
print(a_tag.name, a_tag.attrs, a_tag.string)

a {'id': 'link1', 'href': 'www.3schools.com'} www.3schools.com


In [86]:
name_parents_a_tag = [tag.name for tag in a_tag.parents]
print(name_parents_a_tag)

['p', 'body', 'html', '[document]']


In [87]:
siblings_p_tag = [tag for tag in p_tag.next_siblings]
# siblings_p_tag = list(p_tag.next_siblings)
print(siblings_p_tag)

[<p>This is a paragraph.</p>, <p>
This is another paragraph</p>]


In [88]:
list_id = [tag.attrs["id"] for tag in p_tag.children]
print(list_id)

['link1', 'link2']


## Lấy thông tin trang Web trực tuyến

Ví dụ: Lấy về GDP của tất cả các quốc gia trên thế giới.

- Lấy về mã nguồn trang Web.

In [89]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
html_doc = urlopen(url).read()
soup = BeautifulSoup(html_doc, 'html.parser')

In [90]:
# Lấy tiêu đề trang web
soup.title

<title>List of countries by GDP (nominal) - Wikipedia</title>

- Cần đọc mã nguồn trang Web và phân tích cấu trúc của nó.

In [91]:
table_tag = soup.find("table", attrs = {"class": "wikitable"})
rows = table_tag.find("tbody").find_all("tr")

# rows[1]

In [92]:
# Tại dòng 0, lấy về tiêu đề 3 bảng GDP
table_headers = []
for td in rows[0].find_all("td"):
    table_headers.append(td.b.text.replace('\n', ' ').strip())
    
table_headers

['Per the International Monetary Fund (2019 estimates)',
 'Per the World Bank (2018)',
 'Per the United Nations (2017)']

In [93]:
# Tại dòng 1, lấy về tiêu đề 3 cột dữ liệu
columns = []
for th in rows[1].find("table").find_all("th"):
    columns.append(th.text.replace('\n', ' ').strip())
    
columns

['Rank', 'Country/Territory', 'GDP(US$million)']

In [94]:
# Duyệt qua từng dòng lấy về thông tin tương ứng với cột Ranks, Countrys, GDP
data = {}
for table, header in zip(rows[1].find_all("table"), table_headers):
    table_data = []
    for tr in table.tbody.find_all("tr"):
        row = {}
        for td, th in zip(tr.find_all("td"), columns):
            row[th] = td.text.replace('\n', '').strip()
        table_data.append(row)
    data[header] = table_data

# data
# data['Per the International Monetary Fund (2019 estimates)']
# data.items()

In [95]:
import pandas as pd

df_work_bank = pd.DataFrame(data['Per the World Bank (2018)'])
df_work_bank

,Rank,Country/Territory,GDP(US$million)
0,NaN,NaN,NaN
1,,World,"85,804,391"
2,1,United States,"20,494,100"
3,2,People's Republic of China[n 5],"13,608,152"
4,3,Japan,"4,970,916"
...,...,...,...
188,181,Palau,310
189,182,Marshall Islands,212
190,183,Kiribati,188
191,184,Nauru,115


In [96]:
df_international = pd.DataFrame(data['Per the International Monetary Fund (2019 estimates)'])
df_international

,Rank,Country/Territory,GDP(US$million)
0,NaN,NaN,NaN
1,,World[19],"87,265,226"
2,1,United States,"21,439,453"
3,—,European Union[23][n 1],"18,705,132"
4,2,People's Republic of China[n 2],"14,140,163"
...,...,...,...
190,182,Palau,291
191,183,Marshall Islands,220
192,184,Kiribati,184
193,185,Nauru,108


In [97]:
df_us = pd.DataFrame(data['Per the United Nations (2017)'])
df_us

,Rank,Country/Territory,GDP(US$million)
0,NaN,NaN,NaN
1,,World[25],"80,501,413"
2,1,United States,"19,485,394"
3,2,People's Republic of China[n 5],"12,234,781"
4,3,Japan,"4,872,415"
...,...,...,...
209,189,Marshall Islands,209
210,190,Nauru,182
211,191,Kiribati,180
212,—,Montserrat,63


- Lưu dữ liệu 3 bảng vào 3 tập tin *.csv

In [98]:
df_work_bank.to_csv('Per the World Bank (2018).csv')
df_international.to_csv('Per the International Monetary Fund (2019 estimates).csv')
df_us.to_csv('Per the United Nations (2017).csv')

In [99]:
import csv

# for name, table in data.items():
#     with open(f"{name}.csv", 'w') as out_file:
#         writer = csv.DictWriter(out_file, headers)
#         writer.writeheader()
#         for row in table:
#             if row:
#                 writer.writerow(row)